In [1]:
import h5py
import tensorflow as tf 
import pandas as pd
import numpy as np
from tqdm import tqdm
import math
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

2023-11-28 14:22:04.159220: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
model = tf.keras.models.load_model('../model/RBP_pretrain_1k.h5')
input_layer = model.input 
embed_layer = model.layers[-10].output
embed_model = tf.keras.models.Model(input_layer,embed_layer)


2023-11-28 14:22:06.461571: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 73741 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:07:00.0, compute capability: 8.0


## alternative splicing

In [4]:
rbp_output = h5py.File('../data/alternative_splicing/rbp_splice.h5','w')
file = h5py.File('../data/alternative_splicing/delta_logit.h5','r')
batch_size = 32
pad_size = (1000-400)/2
for label in ('valid','test','train'):
    onehot = file['x_'+label]
    l_seq = onehot[:,:400]
    r_seq = onehot[:,400:]
    l_pad = np.pad(l_seq,((0,0),(math.floor(pad_size),math.ceil(pad_size)),(0,0)))
    r_pad = np.pad(r_seq,((0,0),(math.floor(pad_size),math.ceil(pad_size)),(0,0)))
    l_embed = embed_model.predict(l_pad)
    r_embed = embed_model.predict(r_pad)

    rbp_output.create_dataset(name='xl_'+label,data = np.array(l_embed),dtype = 'float32')
    rbp_output.create_dataset(name='xr_'+label,data = np.array(r_embed),dtype = 'float32')
    rbp_output.create_dataset(name='y_'+label,data = file['y_'+label][:],dtype='float32') 
rbp_output.close()

2023-11-28 14:22:29.063643: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8800


1189/1189 [==============================] - 1s 801us/step


## INSERT-seq

In [7]:
rbp_output = h5py.File('../data/RNAenlong/rbp_embed.h5','w')
file = h5py.File('../data/RNAenlong/insert_dataset.h5','r')
batch_size = 32
pad_size = (1000-file['X_train'].shape[1])/2
for dataset in ['test','train','valid']:
    key = 'X_'+dataset
    onehot = file[key][:]
    pad_onehot = np.pad(onehot,((0,0),(math.floor(pad_size),math.ceil(pad_size)),(0,0)))
    embed = embed_model.predict(pad_onehot)        
    rbp_output.create_dataset(name=key,data = np.array(embed))
    rbp_output.create_dataset(name='Y_'+dataset,data = file['Y_'+dataset][:])
rbp_output.close()

36/36 [==============================] - 0s 3ms/step
